## Cluster Usage Report

This notebook presents an automated analysis of user-level resource consumption on the computing cluster over the past months. The goal is to provide a clear and structured overview of CPU usage trends across users and time periods, enabling better resource planning and transparency.

**Acknowledgments:**  
I would like to sincerely thank my former colleagues **Tobias Esswein** (main developer of these scripts) and **Peter Mlkvik** for their valuable support, input and mentoring during the development of this reporting workflow.


In [ ]:
import numpy as np
import os
import glob
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import dateutil as du
import sqlite3  # for database storage
import plotly.express as px  # for interactive plots

%matplotlib inline  
%config InlineBackend.figure_format = 'png'

In [ ]:
# customize panda options 

def start():
    options = {
        'display': {
            'max_columns': None,
            'max_colwidth': 25,
            'expand_frame_repr': False,  # Don't wrap to multiple pages
            'max_rows': 14,
            'max_seq_items': 50,         # Max length of printed sequence
            'precision': 6,
            'show_dimensions': False
        },
        'mode': {
            'chained_assignment': None   # Controls SettingWithCopyWarning
        }
    }

    for category, option in options.items():
        for op, value in option.items():
            pd.set_option(f'{category}.{op}', value)  # Python 3.6+


start()
del start  # Clean up namespace in the interpreter

In [ ]:
current = '2023-12'

new_report_months = 14

currentMonth = du.parser.parse(current)
pastDates = [currentMonth + du.relativedelta.relativedelta(months=-i) for i in range(24)]
display(pastDates[1:new_report_months])

past1 = currentMonth + du.relativedelta.relativedelta(months=-4)
past2 = currentMonth + du.relativedelta.relativedelta(months=-8)

nodes = '''
  10 regular nodes (24 cores, 64 GB each) valid until 2021-08-31 3
   7 high-performance nodes (36 cores, 192 GB each) valid until 2022-06-30 3
   3 high-performance nodes (36 cores, 192 GB each) valid until 2023-01-31 3
   9 new standard nodes (24 cores, 96 GB each) valid until 2023-01-31 3
   2 big-memory high-throughput nodes (128 cores, 512 GB each) valid until 2024-04-30 3
   2 big-memory high-throughput nodes (128 cores, 512 GB each) valid until 2024-07-31 3
   5 big-memory high-throughput nodes (128 cores, 512 GB each) valid until 2024-10-31 0
   3 medium-memory high-throughput nodes (128 cores, 256 GB each) valid until 2027-01-31 0
'''

nodes = nodes.split('\n')[1:-1]

def getCores(nodes):
    result = []
    for line in nodes:
        
        cores = int(line.split()[0]) * int(line.split('(')[1].split(' ')[0])
        untilDate = du.parser.parse(line.split()[-2])
        bonusMonths = int(line.split()[-1])
        fromDate = untilDate - du.relativedelta.relativedelta(years=4,months=bonusMonths,days=-2)
        fromDate += du.relativedelta.relativedelta(day=1)
        result.append([cores,fromDate,untilDate])
    
    return result

cores = getCores(nodes)

startMonth = currentMonth

num_cores = np.zeros([24])

for m in range(24):
    currentMonth += du.relativedelta.relativedelta(day=31)  
    for core in cores:
        if core[1] <= currentMonth and core[2] >= currentMonth:
            num_cores[m] += core[0]
    currentMonth += du.relativedelta.relativedelta(months=-1)  

num_cores = np.flipud(num_cores)

display(startMonth)
display(num_cores)

In [ ]:
months = [startMonth + du.relativedelta.relativedelta(months=-i) for i in reversed(range(24))]
df_capacity = pd.DataFrame({'Month': [m.strftime('%Y-%m') for m in months],
                            'CoresAvailable': num_cores})

# Connect & store
conn = sqlite3.connect('cluster_usage.db')
df_capacity.to_sql('capacity', conn, if_exists='replace', index=False)
conn.close()

display(df_capacity)

In [ ]:
## cost per CPU hour
cph = 12000/364/365/24
## cost per CPU month
cpm = 12000/36/4/12

## electricity per coremonth (kWh)
epcm = 10 * (24*30) / 1000 # Watt * hours / 1000

## cost per kWh
cpkw = 0.28  # CHF / kWh

## CO2 per kWh
co2pkw = 28  # g / kWh

## electricity cost per coremonth
ecpcm = epcm * cpkw

## CO2 emissions per coremonth
co2pcm = epcm * co2pkw

print('{:.0f} CO2 / coremonth'.format(round(co2pcm/10)*10))

In [ ]:
sns.set()
cmap = sns.color_palette(n_colors=20)
# cmap = sns.diverging_palette(250, 150, s=75, l=40, n=10, center="light")
new_cmap = cmap
for k,el in enumerate(cmap):
    if k>=10:
        new_cmap[k] = np.array(el)*0.4+0.6
    else:
        new_cmap[k] = np.array(el)

In [ ]:
def create_fig(dpi):
    ## set properties for figure
    sns.set(style="white")
    font_size = 12
    mpl.rcParams['ytick.labelsize'] = font_size
    mpl.rcParams['xtick.labelsize'] = font_size
    mpl.rcParams['axes.labelsize'] = font_size
    mpl.rcParams['font.size'] = font_size
    mpl.rcParams['axes.titlesize'] = font_size
    mpl.rcParams['legend.title_fontsize'] = font_size * 0.8   
    mpl.rcParams['legend.fontsize'] = font_size * 0.7   
    ## create figure with golden ratio width/height
    figure_width = 16   # cm
    fig = plt.figure(figsize=(21.0/2.54, 29.7/2.54), dpi = dpi)
    ## add one subplot
    return fig

In [ ]:
df = pd.read_fwf('./hpc_accounting/'+pastDates[0].strftime('%Y-%m')+'.txt',
                      colspecs=[(0,40),(49,57),],header=0,dtype={0: str, 1: np.float64})
df.set_index('User',inplace=True)
df.rename(columns = {'Cores':pastDates[0].strftime('%Y-%m')}, inplace = True)
df.dropna(inplace=True)


# import for reports from 2022-09 to now
for pastDate in pastDates[1:new_report_months]:
    # print(f'new {pastDate}')
    df1 = pd.read_fwf('./hpc_accounting/'+pastDate.strftime('%Y-%m')+'.txt',
                      colspecs=[(0,40),(49,57),],header=0,dtype={0: str, 1: np.float64})
    df1.set_index('User',inplace=True)
    df1.rename(columns = {'Cores':pastDate.strftime('%Y-%m')}, inplace = True)
    df1.dropna(inplace=True)
#     display(df1)
    df = df.join(df1, how='outer')
    
del df1

## import for reports from xxxx-xx to 2022-09
for pastDate in pastDates[new_report_months:]:
    # print(f'old {pastDate}')
    df1 = pd.read_fwf('./hpc_accounting/'+pastDate.strftime('%Y-%m')+'.txt',
                      colspecs=[(0,40),(49,57),],header=0,dtype={0: str, 1: np.float64})
    df1.set_index('User',inplace=True)
    df1.rename(columns = {'Cores':pastDate.strftime('%Y-%m')}, inplace = True)
    df1.dropna(inplace=True)
#     display(df1)
    df = df.join(df1, how='outer')

## flip dataframe left to right
columns = df.columns.tolist()
columns = columns[::-1]
df = df[columns]
# totals = df.loc['TOTAL']
df.drop('TOTAL',inplace=True)

## calculate mean usage
mean_usage_3 = df.sum().rolling(3).mean()
# display(mean_usage_3)
mean_usage_12 = df.sum().rolling(12).mean()

## drop every row/name with less than 0.1 nodehours in selected period
df = df.iloc[:,-12:]
df['total'] = df.T.sum()
df.fillna(0,inplace=True)
df = df[df.total > 0.1]

display(df)

#LUCA: I think these are corehours and not coremonths
coremonths_total = df.loc[:,'total'].sum()
coremonths_lastmonth = df.iloc[:,-2].sum()
display(coremonths_total)

In [ ]:
all_usage = []
for i, pastDate in enumerate(pastDates):
    
    fname = f'./hpc_accounting/{pastDate.strftime("%Y-%m")}.txt'
    colspecs = [(0, 40), (49, 57)]

    df1 = pd.read_fwf(fname,
                      colspecs=colspecs,
                      header=0,
                      names=['User', 'Cores'],
                      dtype={'User': str, 'Cores': np.float64})
    df1.dropna(inplace=True)
    df1 = df1[~df1['User'].str.contains('TOTAL')]
    df1['Month'] = pastDate.strftime('%Y-%m')
    all_usage.append(df1)


In [ ]:
df_usage = pd.concat(all_usage, ignore_index=True)

# Clean possible extra whitespace in usernames
df_usage['User'] = df_usage['User'].str.strip()

In [ ]:
conn = sqlite3.connect('cluster_usage.db')
df_usage.to_sql('usage', conn, if_exists='replace', index=False)
conn.close()

display(df_usage.head())


In [ ]:
conn = sqlite3.connect('cluster_usage.db')

df_monthly = pd.read_sql("""
SELECT Month, SUM(Cores) as TotalCores
FROM usage
GROUP BY Month
ORDER BY Month
""", conn)

conn.close()

df_monthly['Mean3'] = df_monthly['TotalCores'].rolling(3).mean()
df_monthly['Mean12'] = df_monthly['TotalCores'].rolling(12).mean()

display(df_monthly)


In [ ]:
conn = sqlite3.connect('cluster_usage.db')

df_user_total = pd.read_sql("""
SELECT User, SUM(Cores) as TotalCores
FROM usage
WHERE Month >= DATE('now', '-12 months')
GROUP BY User
HAVING TotalCores > 0.1
""", conn)

conn.close()

display(df_user_total)


In [ ]:
# display(df.iloc[:,-1])
weights = [50,30,20,0,0,0]
df['sort'] = (
    df.iloc[:,-6]/max(df.iloc[:,-6])*weights[5]/sum(weights) +
    df.iloc[:,-5]/max(df.iloc[:,-5])*weights[4]/sum(weights) +
    df.iloc[:,-4]/max(df.iloc[:,-4])*weights[3]/sum(weights) +
    df.iloc[:,-3]/max(df.iloc[:,-3])*weights[2]/sum(weights) +
    df.iloc[:,-2]/max(df.iloc[:,-2])*weights[1]/sum(weights) +
    df.iloc[:,-1]/max(df.iloc[:,-1])*weights[0]/sum(weights)  
)
df.sort_values(by='sort',ascending=False, inplace=True)

# LUCA: incredibly nice sorting procedure..!
display(df.head(5))
df.drop_duplicates(inplace=True)
df.drop('sort',axis=1,inplace=True)
display(df)

In [ ]:
user_mean_one_year = np.average(df['total'].values)
user_sigma_one_year = np.std(df['total'].values)
print(user_mean_one_year)
print(user_sigma_one_year)

In [ ]:
CHF_per_CPU_month = 4.10

In [ ]:
cost_one_year = user_mean_one_year * CHF_per_CPU_month
cost_one_year

In [ ]:
fig = create_fig(120)

gs1 = mpl.gridspec.GridSpec(2,2)
gs1.update(wspace=0.2, hspace=0.0) # set the spacing between axes.
# ax1 = plt.subplot(gs1[0, :-1])
# ax2 = plt.subplot(gs1[0, -1:])
ax1 = plt.subplot(gs1[:1,:])
ax2 = plt.subplot(gs1[1:,:1])
ax3 = plt.subplot(gs1[1:, 1:])

# display(df.T)

## combine small users to others
dfplot = df.iloc[:17]
others = df.iloc[17:].sum()
others.name = 'Others'
dfplot = pd.concat([dfplot, others], ignore_index=True)


## uncomment for stacked bar plot
# dfplot.T.iloc[:-1,:].plot.bar(ax=ax1,color=cmap,legend=False,alpha=0.9, stacked=True, width=.618)

## uncomment for stacked area plot
dfplot.T.iloc[:-1,:].plot.area(ax=ax1,color=cmap,legend=False,alpha=0.8,)

ax1.set_xlim([0,11])

# LUCA: the number of cores in the various node typologies etc... are needed to plot the (varying) quota..!
ax1.plot(np.array(dfplot.T.iloc[:-1,:].index)[0:-1],num_cores[-12:],color='k',alpha=0.8,linestyle='-')
ax1.plot(np.array(dfplot.T.iloc[:-1,:].index)[0:-1],mean_usage_3[-12:].values,color='w',alpha=0.8,linestyle='-')
ax1.plot(np.array(dfplot.T.iloc[:-1,:].index)[0:-1],mean_usage_3[-12:].values,color='k',alpha=0.8,linestyle='--')
ax1.plot(np.array(dfplot.T.iloc[:-1,:].index)[0:-1],mean_usage_12[-12:].values,color='w',alpha=0.8,linestyle='-')
ax1.plot(np.array(dfplot.T.iloc[:-1,:].index)[0:-1],mean_usage_12[-12:].values,color='k',alpha=0.8,linestyle=':')
ax1.set_title('CLUSTER usage ({})\n'.format(dfplot.columns[-2]), fontdict={'fontsize':24})
ax1.set_ylabel('# cores')
ax1.set_xticks(np.arange(0,12,1))
ax1.set_xticklabels(dfplot.T.index[:12], rotation = 45)
# print(dfplot.T.index)
ax1.grid()

## total usage pie plot
df_pie_tot = dfplot.iloc[:,-1]
df_pie_tot.plot.pie(ax=ax2, labels=[''.format(x) for x in df_pie_tot.values],colors=cmap, startangle=90)#, autopct='%2.0f')

## last month usage pie plot
df_pie = dfplot.iloc[:,-2]
# df_pie = df_pie.append(pd.Series(data=dfplot.iloc[9:,-2].sum(), index=['Other']))
# display(df_pie)
df_pie.plot.pie(ax=ax3, labels=[''.format(x) for x in df_pie.values], colors=cmap, startangle=90, radius=1)#, autopct='%2.0f')

#draw circle
centre_circle = plt.Circle((0,0),0.75,fc='white')
ax2.add_artist(centre_circle)
ax2.axis('equal')
centre_circle = plt.Circle((0,0),0.75,fc='white')
ax3.add_artist(centre_circle)
ax3.axis('equal')

ax2.legend(df_pie.index, bbox_to_anchor=(1.06,-0.06),ncol=3,loc='center', fontsize=11)
ax2.set_ylabel('')
ax2.text(0,0,'past year', horizontalalignment='center',verticalalignment='center', fontsize=14)


ax3.set_ylabel('')
ax3.text(0,0,'past month', horizontalalignment='center',verticalalignment='center', fontsize=14)

ax1.text(dfplot.T.iloc[:-1,:].index[-1],mean_usage_3[-1],' 3m.', horizontalalignment='left',verticalalignment='center', fontsize=14)
ax1.text(dfplot.T.iloc[:-1,:].index[-1],num_cores[-1],' quota', horizontalalignment='left',verticalalignment='center', fontsize=14)
ax1.text(dfplot.T.iloc[:-1,:].index[-1],mean_usage_12[-1],' 12m.', horizontalalignment='left',verticalalignment='center', fontsize=14)

ax1.set_ylim(0,5000)

plt.savefig('./hpc_accounting/cluster_'+current+'.pdf', dpi=300)#, bbox_inches='tight')
plt.savefig('./hpc_accounting/cluster_'+current+'.png', dpi=300)#, bbox_inches='tight')